In [ ]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
import sklearn
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score
import glob

In [ ]:
# Cài kaggle
!pip install -q kaggle
!pip install pyyaml h5py  # Required to save models in HDF5 format
# tạo file kaggle.json
!touch kaggle.json
!echo '{"username":"lasion","key":"241a4d93286f792dff44b5da573987ad"}' > kaggle.json
# đăng nhập kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

!kaggle datasets download -d abhinavkrjha/dog-breed-classification

!unzip /content/dog-breed-classification.zip -d train

ref                                                    title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
thedrcat/daigt-v2-train-dataset                        DAIGT V2 Train Dataset                       29MB  2023-11-16 01:38:36           2170        205  1.0              
thedrcat/daigt-proper-train-dataset                    DAIGT Proper Train Dataset                  119MB  2023-11-05 14:03:25           1952        158  1.0              
muhammadbinimran/housing-price-prediction-data         Housing Price Prediction Data               763KB  2023-11-21 17:56:32          10045        170  1.0              
carlmcbrideellis/llm-7-prompt-training-dataset         LLM: 7 prompt training dataset               41MB  2023-11-15 07:32:56           1776     

In [ ]:
DATASET_PATH = '/content/'
N_CATEGORIES = 14
CATEGORIES_TRAIN = os.listdir(os.path.join(DATASET_PATH, 'train'))
ORIGINAL_DIMENSIONS = (100, 100, 3)
RESIZED_DIMENSIONS = (50, 50, 3)
print(CATEGORIES_TRAIN)

['Afgan Hound dog', 'Alaskan Malamute dog', 'Boxer dog', 'Beagle dog', 'Borzoi dog', 'Auggie dog', 'Bugg dog', 'Bichon Frise dog', 'Affenhuahua dog', 'Bocker dog', 'Akita dog', 'American Bulldog dog', 'Belgian Tervuren dog', 'Bulldog dog']


In [ ]:
def load_images(path, categories):
    flat_data_arr = []
    target_arr = []

    for category in categories:
        print(f'Loading... Category: {category}')

        image_files = glob.glob(os.path.join(path, category, '*.jpg'))

        for img_file in image_files:
            img_array = imread(img_file)
            img_gray = rgb2gray(img_array)  # Chuyển đổi ảnh sang độ xám
            img_resized = resize(img_gray, RESIZED_DIMENSIONS)
            flat_data_arr.append(img_resized.flatten())
            target_arr.append(category)

        print(f'Loaded category: {category} successfully')

    flat_data = np.array(flat_data_arr)
    target = np.array(target_arr)

    return flat_data, target

In [ ]:
train_flat_data, train_target = load_images(os.path.join(DATASET_PATH, 'train'), CATEGORIES_TRAIN)

df_train = pd.DataFrame(train_flat_data)
df_train['Target'] = train_target

x_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]

Loading... Category: Afgan Hound dog
Loaded category: Afgan Hound dog successfully
Loading... Category: Alaskan Malamute dog
Loaded category: Alaskan Malamute dog successfully
Loading... Category: Boxer dog
Loaded category: Boxer dog successfully
Loading... Category: Beagle dog
Loaded category: Beagle dog successfully
Loading... Category: Borzoi dog
Loaded category: Borzoi dog successfully
Loading... Category: Auggie dog
Loaded category: Auggie dog successfully
Loading... Category: Bugg dog
Loaded category: Bugg dog successfully
Loading... Category: Bichon Frise dog
Loaded category: Bichon Frise dog successfully
Loading... Category: Affenhuahua dog
Loaded category: Affenhuahua dog successfully
Loading... Category: Bocker dog
Loaded category: Bocker dog successfully
Loading... Category: Akita dog
Loaded category: Akita dog successfully
Loading... Category: American Bulldog dog
Loaded category: American Bulldog dog successfully
Loading... Category: Belgian Tervuren dog
Loaded category: B

In [ ]:
import os

train_dir = '/content/train'  # Đường dẫn đến thư mục train

# Lấy danh sách tên các thư mục trong thư mục train
subdirectories = [name for name in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, name))]
print(subdirectories)

categories = sorted(subdirectories)  # Sắp xếp danh sách các thư mục theo thứ tự

num_categories = len(categories)  # Số lớp
num_subcategories = num_categories // 2  # Số tầng con, là nửa số lớp

subcategories = []
start = 0
for i in range(num_subcategories):
    end = start + (num_categories - start) // (num_subcategories - i)
    subcategory = categories[start:end]
    subcategories.append(subcategory)
    start = end

# Hiển thị các tầng con
for i, subcategory in enumerate(subcategories):
    print(f"Tầng con {i+1}: {subcategory}")

['Afgan Hound dog', 'Alaskan Malamute dog', 'Boxer dog', 'Beagle dog', 'Borzoi dog', 'Auggie dog', 'Bugg dog', 'Bichon Frise dog', 'Affenhuahua dog', 'Bocker dog', 'Akita dog', 'American Bulldog dog', 'Belgian Tervuren dog', 'Bulldog dog']
Tầng con 1: ['Affenhuahua dog', 'Afgan Hound dog']
Tầng con 2: ['Akita dog', 'Alaskan Malamute dog']
Tầng con 3: ['American Bulldog dog', 'Auggie dog']
Tầng con 4: ['Beagle dog', 'Belgian Tervuren dog']
Tầng con 5: ['Bichon Frise dog', 'Bocker dog']
Tầng con 6: ['Borzoi dog', 'Boxer dog']
Tầng con 7: ['Bugg dog', 'Bulldog dog']


In [ ]:
models = []
for layer in subcategories:
    layer_df = df_train[df_train['Target'].isin(layer)]
    x_layer_train = layer_df.iloc[:, :-1]
    y_layer_train = layer_df.iloc[:, -1]

    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [0.0001, 0.001, 0.1, 1],
        'kernel': ['rbf', 'poly'],
    }
    svc = svm.SVC(probability=True)
    model = GridSearchCV(svc, param_grid)
    print('Training model...')
    model.fit(x_layer_train, y_layer_train)

    models.append(model)

# Phân loại kết hợp (Ensemble-based classification)
from sklearn.ensemble import VotingClassifier
ensemble_model = VotingClassifier([(str(i), model) for i, model in enumerate(models)], voting='soft')
ensemble_model.fit(x_train, y_train)

# Lưu model đã huấn luyện
model_path = '/content/trained_model_archive14444.pkl'
import joblib
joblib.dump(ensemble_model, model_path)

Training model...
Training model...
Training model...
Training model...
Training model...
Training model...
Training model...


['/content/trained_model_archive14444.pkl']

In [ ]:
CATEGORIES_TEST = os.listdir(os.path.join(DATASET_PATH, 'train'))
# Đường dẫn đến mô hình đã lưu
model_path = '/content/trained_model_archive14444.pkl'

# Load mô hình đã train
import joblib
trained_model = joblib.load(model_path)

# Dự đoán nhãn cho dữ liệu kiểm tra
# Tải dữ liệu kiểm tra
test_flat_data, test_target = load_images(os.path.join(DATASET_PATH, 'train'), CATEGORIES_TEST)

df_test = pd.DataFrame(test_flat_data)
df_test['Target'] = test_target

x_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]
y_pred = trained_model.predict(x_test)

# Tính toán các độ đo đánh giá
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)

Loading... Category: Afgan Hound dog
Loaded category: Afgan Hound dog successfully
Loading... Category: Alaskan Malamute dog
Loaded category: Alaskan Malamute dog successfully
Loading... Category: Boxer dog
Loaded category: Boxer dog successfully
Loading... Category: Beagle dog
Loaded category: Beagle dog successfully
Loading... Category: Borzoi dog
Loaded category: Borzoi dog successfully
Loading... Category: Auggie dog
Loaded category: Auggie dog successfully
Loading... Category: Bugg dog
Loaded category: Bugg dog successfully
Loading... Category: Bichon Frise dog
Loaded category: Bichon Frise dog successfully
Loading... Category: Affenhuahua dog
Loaded category: Affenhuahua dog successfully
Loading... Category: Bocker dog
Loaded category: Bocker dog successfully
Loading... Category: Akita dog
Loaded category: Akita dog successfully
Loading... Category: American Bulldog dog
Loaded category: American Bulldog dog successfully
Loading... Category: Belgian Tervuren dog
Loaded category: B